In [ ]:
!python -V

In [ ]:
!pip install pandas==1.2.5 pandas_datareader yfinance==0.1.62 ipywidgets pandas_market_calendars matplotlib numpy pycorrelate


In [ ]:
import os
from datetime import datetime
import concurrent
import pandas_datareader.data as web
import pandas as pd
import datetime
import concurrent.futures
from concurrent.futures import wait, ALL_COMPLETED
import yfinance as yf
import urllib
import urllib.request
import time
from datetime import timedelta
from finquant.portfolio import build_portfolio

from ipywidgets import interactive
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import pycorrelate as pyc
import seaborn as sns
from yahoofinancials import YahooFinancials

import pandas_market_calendars as mcal
import re
from sklearn.model_selection import train_test_split

from scipy import stats # For in-built method to get PCC
from scipy.ndimage.interpolation import shift

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
#import statsmodels.formula.api as sm
import scipy.stats  as stats
from itertools import cycle
import matplotlib.pyplot as plt

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):

    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y)

def crosscorrelation(x, y, maxlag, mode='corr'):
	"""
	Cross correlation with a maximum number of lags.

	`x` and `y` must be one-dimensional numpy arrays with the same length.

	This computes the same result as
		numpy.correlate(x, y, mode='full')[len(a)-maxlag-1:len(a)+maxlag]

	The return vaue has length 2*maxlag + 1.
	"""
	py = np.pad(y.conj(), 2*maxlag, mode='constant')
	T = np.lib.stride_tricks.as_strided(py[2*maxlag:], shape=(2*maxlag+1, len(y) + 2*maxlag),
				   strides=(-py.strides[0], py.strides[0]))
	px = np.pad(x, maxlag, mode='constant')
	if mode == 'dot':       # get lagged dot product
		return T.dot(px)
	elif mode == 'corr':    # gets Pearson correlation
		return (T.dot(px)/px.size - (T.mean(axis=1)*px.mean())) / \
			   (np.std(T, axis=1) * np.std(px))

def unique(list1):

    # intilize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

    return(unique_list)

def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
        
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [ ]:
#frequency = output_variable.value
frequency = "Q"

w=52*12
#start = datetime.datetime(2010, 1, 1)

#end = datetime.datetime(2013, 1, 27)
end_date = datetime.date.today()

start_date = end_date - timedelta(weeks=w)

pd.set_option('display.max_columns', None) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', None) #replace n with the number of rows you want to see completely

#cores = int(len(os.sched_getaffinity(0)))

print(end_date)
print(start_date)

one_week_end = end_date - 5 * pd.tseries.offsets.BDay()
one_week_end = one_week_end.strftime("%Y-%m-%d")

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Show available calendars
print(mcal.get_calendar_names())
early = nyse.schedule(start_date, end_date)

nstocks = 300

In [ ]:
url = "ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqtraded.txt"

urllib.request.urlretrieve(url, "nasdaqtraded.txt")
urllib.request.urlretrieve(url, "mfundslist.txt")
urllib.request.urlretrieve(url, "bonds.txt")

df1 = pd.read_csv("nasdaqtraded.txt", sep="|")[0:-1]
df2 = pd.read_csv("mfundslist.txt", sep="|")[0:-1]
df3 = pd.read_csv("bonds.txt", sep="|")[0:-1]

# combined = pd.concat([df1['Symbol'],df2['Symbol'],df3['Symbol']],axis=0)

# process symbols for bad characters
BAD_CHARS = ["$", "."]
# pat = '|'.join(['({})'.format(re.escape(c)) for c in BAD_CHARS])
# cleaned = unique(combined.replace(BAD_CHARS,'-'))

# choose size
size = nstocks
# stocks = list(df1["Symbol"].sample(n=int(size/3)))
stocks = list(
    df1["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
mfunds = list(
    df2["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
bonds = list(
    df3["Symbol"]
    .replace(".", "-")
    .replace("\\$", "-P", regex=True)
    .sample(n=int(size / 3))
)
symbols = list(set(stocks + mfunds + bonds))  # unique(stocks + mfunds + bonds)
# symbols = unique(stocks)

In [ ]:
pf_pre = build_portfolio(
    names=symbols, start_date=start_date, end_date=one_week_end, data_api="yfinance"
)


In [ ]:
# Show available calendars
# print(mcal.get_calendar_names())

vetted_symbols = list(
    pf_pre.data.loc[
        (np.intersect1d(list(pf_pre.data.index.strftime("%Y-%m-%d")), early.index.strftime('%Y-%m-%d')))
    ]
    .head(-1)
    .tail(-1)
    .dropna(axis=1)
    .columns
)

In [ ]:

#Ultra-Low-Sulfur No. 2 Diesel Fuel Prices: Los Angeles (WDFUELLA)
#US Regular All Formulations Gas Price (GASREGW)
#Trade Weighted U.S. Dollar Index: Broad, Goods and Services (DTWEXBGS)


etf_commodities = ['DBO','CORN', 'WEAT', 'SOYB', 'JO', 'SGG', 'BAL', 'COW', 'MOO', 'TAGS', 'KOL' ]
#Gold, Silver, Platinum, Copper, Paladium, Aluminum, Iron, Steel
etf_metals = ['IAU', 'SLV', 'PGM', 'JJC', 'PALL', 'JJU', 'IFUNX', 'SLX']
#US dollar, European Euro, Japanese yen, Pound sterling, Australian dollar, Canadian dollar, Swiss franc, Chinese Yuan Renminbi, Swedish Krona, Peso, India
#defunct: Russia: XRU, Mexico: FXM
etf_foreign_exchanges = ['UUP','FXE','FXY','FXB','FXA','FXC','FXF','CYB', 'FXS', 'INR']
#residential, Ishares all NAmerica
etf_real_estate = ['REZ', 'IYR']
#Russia, Germany, UK, Japan, China, Euro, Euro, Brazil, Latin America, Mexico, India
etf_economies = ['ERUS','EWG','EWU','EWJ','MCHI','EZU','IEUR','EWZ','ILF','EWW','INDA']
#Ishares Investment Grade, IShares core aggregate Investment grade, Short, Total, 1-5 Years, 5-10 Years, 10 Years, Gov/Credit
#defunct:

etf_spdr_indexes = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
etf_dow_components = ['MMM','AXP','AMGN','AAPL','BA','CAT','CVX','CSCO','KO','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PG','CRM','TRV','UNH','VZ','V','WMT','WBA','DIS']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

crypto = ['BTC-USD','ETH','RPL','BCH','EOS','LTC']

#,'GOLDAMGBD228NLBM',
FRED_Indicators = ['T10YIE','WDFUELLA','DTWEXBGS','GASREGW','DGS2','CPALTT01USQ657N','PAYEMS','MSPUS','ASPUS','IRLTLT01USM156N','MEPAINUSA672N','MABMM301USM189S','LFWA64TTUSM647S','MANMM101USA189S','MICH','UMCSENT','CSCICP03USM665S','DGS10','DTB3','DGS3MO','CASTHPI','GDPC1','CIVPART','POPTOTUSA647NWDB','MEHOINUSA672N','HOSMEDUSM052N','MORTGAGE30US','TTLHH','CSUSHPINSA','EMRATIO','CPIAUCSL','PSAVERT','LRUN64TTUSQ156S','USSTHPI','NYSTHPI','M2V','GFDEBTN','DFII10','GFDEGDQ188S','CUSR0000SEHA','ETOTALUSQ176N','ERENTUSQ176N','RECPROUSM156N','T5YIFR','BAMLHYH0A0HYM2TRIV','BAMLCC0A1AAATRIV','GVZCLS','DGS1','BAMLCC0A4BBBTRIV','VXVCLS','IC4WSA','WILLMICROCAPPR','WILLLRGCAPVAL','CFNAIDIFF','MZMSL','KCFSI','T5YIE','TOTALSA','USSLIND','AWHAETP','CES0500000003','TCU','WTB3MS','WGS3MO','TWEXB','DEXCHUS','DEXUSUK','CILACBQ158SBOG','CES4348400001','FEDFUNDS','TDSP','PERMIT','CP','PRFI','DRSFRMACBS','DRCCLACBS','DRBLACBS','DALLCIACBEP','USROA','USROE','RSAHORUSQ156S','MEFAINUSA672N','COMREPUSQ159N','HDTGPDUSQ163N','POP','NROU','FGCCSAQ027S','TEDRATE', 'VIXCLS', 'NFCI','INDPRO','LES1252881600Q','CUUR0000SEHA','LEU0252918500Q','BAA10Y','BAMLC0A0CM','BAMLH0A3HYC','BOGMBASE','DCOILBRENTEU','DCOILWTICO','DFF','DGS1MO','DGS30','DGS5','FPCPITOTLZGUSA','ICSA','INTDSRUSM193N','M1','M1V','MPRIME','PPIACO','SPCS20RSA','STLFSI2','T10Y2Y','T10Y3M','TB3MS','TREAST','UNRATE','WPU0911']
#FRED_Indicators = ["CPALTT01USQ657N","PAYEMS","IRLTLT01USM156N","MABMM301USM189S","LFWA64TTUSM647S","MANMM101USA189S","MICH","UMCSENT","CSCICP03USM665S","DGS10","DTB3","DGS3MO","CASTHPI","GDPC1","CIVPART","POPTOTUSA647NWDB","MEHOINUSA672N","HOSMEDUSM052N","MORTGAGE30US","TTLHH","CSUSHPINSA","EMRATIO","CPIAUCSL","PSAVERT","LRUN64TTUSQ156S","USSTHPI","NYSTHPI","M2V","GFDEBTN","DFII10","GFDEGDQ188S","CUSR0000SEHA","ETOTALUSQ176N","ERENTUSQ176N","RECPROUSM156N","T5YIFR","BAMLHYH0A0HYM2TRIV","BAMLCC0A1AAATRIV","GVZCLS","DGS1","BAMLCC0A4BBBTRIV","VXVCLS","IC4WSA","WILLMICROCAPPR","WILLLRGCAPVAL","CFNAIDIFF","MZMSL","KCFSI","T5YIE","TOTALSA","USSLIND","AWHAETP","CES0500000003","TCU","WTB3MS","WGS3MO","TWEXB","DEXCHUS","DEXUSUK","CILACBQ158SBOG","CES4348400001","FEDFUNDS","TDSP","PERMIT","GFDEGDQ188S","CP","PRFI","DRSFRMACBS","DRCCLACBS","DRBLACBS","DALLCIACBEP","USROA","USROE","RSAHORUSQ156S","MEFAINUSA672N","COMREPUSQ159N","HDTGPDUSQ163N","POP","NROU","FGCCSAQ027S","TEDRATE", "VIXCLS", "NFCI","INDPRO","LES1252881600Q","CUUR0000SEHA","LEU0252918500Q","BAA10Y","BAMLC0A0CM","BAMLH0A3HYC","BOGMBASE","DCOILBRENTEU","DCOILWTICO","DFF","DGS1MO","DGS30","DGS5","FPCPITOTLZGUSA","GOLDAMGBD228NLBM","ICSA","INTDSRUSM193N","M1","M1V","MPRIME","PPIACO","SPCS20RSA","STLFSI2","T10Y2Y","T10Y3M","TB3MS","TREAST","UNRATE","WPU0911"]

Indexes = ['^SP500TR', '^GSPC', 'QQQ', 'DIA', 'VTWO']

ManualStocks = ['VOO','SPY']
ManualStocks.extend(vetted_symbols)

etf_indexes_and_Crypto_list = [Indexes, ManualStocks, etf_commodities, etf_metals, etf_foreign_exchanges, etf_real_estate, etf_economies, etf_bonds, etf_muni_bonds, etf_treasuries, crypto, etf_spdr_indexes, etf_dow_components]

commodities = []
for sublist in etf_indexes_and_Crypto_list:
    for val in sublist:
        commodities.append(val)

#pool2 = concurrent.futures.ProcessPoolExecutor(cores)

completed = []
def dl(name):
    subset = yf.download(name, start=start_date, end=end_date, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[start_date.strftime('%Y-%m-%d'):end_date.strftime('%Y-%m-%d')]
    #sleep(4)
    if len(subset) != 0:
        completed.append(name)
        return (subset)
    
def dl2(assets):
    #https://webcache.googleusercontent.com/search?q=cache:Em9Ge5B9ue8J:https://stackoverflow.com/questions/62614282/is-the-yfinance-module-broken-i-keep-getting-a-jsondecodeerror+&cd=3&hl=en&ct=clnk&gl=us

    yahoo_financials = YahooFinancials(assets)

    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)


#futures2 = [pool2.submit(dl, args) for args in commodities]
#wait(futures2, timeout=4, return_when=ALL_COMPLETED)
futures2 = []
#'''
for i in commodities:
    futures2.append(dl(i))
#'''

'''
ohlcv_data = dl2(commodities)
for i in commodities:
    subset = pd.DataFrame(ohlcv_data[i]['prices']).set_index(['formatted_date'])[['open','high','low','close','adjclose','volume']].dropna()
    #sleep(4)
    if len(subset) != 0:
        completed.append(i)
        futures2.append(subset)
'''
#print(futures2.describe())


In [ ]:
if(type(futures2[31]) == type(None)):
    print("None")
else:
    print("Not")
    
type(futures2[31])  

In [ ]:
commodities_ = pd.DataFrame()

#commodities_includes = []
names = []

#completed_2 = []

for x in range(0,len(completed)):
    values = futures2[x]
    
    if(type(values) == type(None)):
        #commodities_includes.append(False)
        print("None")
    else:
        #print("Not")
        #commodities_includes.append(True)
        values.index = pd.to_datetime(values.index)
        values = values.resample(frequency).mean().dropna()
        values['Symbol'] = x
        values = values.loc[~values.index.duplicated(keep='last')]
        values = values.reset_index()
        names.append(completed[x])

        commodities_ = pd.concat([commodities_,values], axis=0)

In [ ]:
#type(commodities_includes)

#np.where(type(commodities_includes) == type(None), False, True)

In [11]:
pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol']).describe()

Symbol,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222
count,49.000000,49.000000,49.000000,49.000000,46.000000,46.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,4.900000e+01,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,47.000000,42.000000,42.000000,16.000000,16.000000,16.000000,16.000000,49.000000,40.000000,46.000000,49.000000,49.000000,16.000000,16.000000,48.000000,16.000000,10.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,42.000000,43.000000,49.000000,49.000000,45.000000,49.000000,49.000000,49.000000,44.000000,49.000000,31.000000,49.000000,49.000000,49.000000,40.000000,49.000000,49.000000,33.000000,31.000000,37.000000,21.000000,49.000000,49.000000,49.000000,49.000000,28.000000,49.000000,40.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,30.000000,36.000000,49.000000,49.000000,49.000000,15.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,25.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,12.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,4286.340581,2234.025413,131.776243,177.116001,48.569494,195.513254,205.407567,35.294122,18.504532,2.548709,21.969975,75.308895,92.395017,6.458344,27.388440,43.453384,915.256872,10.130047,158.251994,40.897770,4.152772,92.930186,9.209708,16.607430,38.824252,11.185908,64.400053,32.527656,22.504171,85.621097,52.452151,11.192434,27.942581,1.870321,41.653953,19.934517,6.744423,620.486621,97.536420,32.615564,30.965579,79.987373,21.640597,49.795801,2.381957,66.950783,26.636337,4.029760,7.406827,10.757222,22.268775,19.038295,98.879884,29.560000,44.507654,7.381863,23.593837,3.105889,1.927256,28.656264,10.738247,50.898616,32.806157,3.743616,21.687052,46.381656,9.880435,30.079336,33.804356,2.056786,16.496241,95.779841,13.775945,42.283096,25.473971,16.092259,5.597961,9.113969,40.991889,10.911405,26.730719,36.010366,1.087748e+10,9.529975,37.550859,38.142034,12.757521,68.649792,3.428474,316.719085,32.236800,31.183321,2.871659,32.482336,38.180730,39.176754,5.570341,2.819435,73.35936

In [12]:
#yahoofinancials
#commodities_pvt = pd.pivot_table(commodities_, values='close', index=['formatted_date'],columns=['Symbol'])

#yfinance
commodities_pvt = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
commodities_pvt.columns = names
wd = os.getcwd()

commodities_pvt.to_csv(re.sub('code', 'data', wd)+"\commodities.csv")

In [13]:
def Fred_Data(name):
    temp = web.DataReader(str(name), 'fred', start_date, end_date)
    temp.index = pd.to_datetime(temp.index)
    temp = temp.resample(frequency).mean().dropna()
    return(temp)

#pool1 = concurrent.futures.ProcessPoolExecutor(cores)

#futures1 = [pool1.submit(Fred_Data, args) for args in FRED_Indicators]
#wait(futures1, timeout=None, return_when=ALL_COMPLETED)

FRED_set = []
FRED_completed = []
for i in FRED_Indicators:
    FRED_completed.append(i)
    FRED_set.append(Fred_Data(i))

RemoteDataError: Unable to read URL: https://fred.stlouisfed.org/graph/fredgraph.csv?id=MSUS
Response Text:
b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1">\r\n    <title>Error - St. Louis Fed</title>\r\n    <meta name="description" content="">\r\n    <meta name="keywords" content="">\r\n    <link rel="stylesheet" type="text/css" href="/css/bootstrap.custom.min.css?1551876305">\r\n    <link rel="stylesheet" type="text/css" href="/css/home.min.css?1553087253">\r\n    <link rel="stylesheet" type="text/css" href="/assets/fontawesome-free/css/all.min.css">\r\n    <link rel="stylesheet" type="text/css" href="/assets/select2/dist/css/select2.min.css">\r\n    <style>p {\r\n        margin-bottom: 1.5em;\r\n    }</style>\r\n</head>\r\n<body>\r\n<link rel="preconnect" href="https://fonts.gstatic.com">\n<link href="https://fonts.goog

In [ ]:

FRED_pvt = pd.DataFrame()

for x in range(0,len(FRED_completed)):
    values = FRED_set[x]
    #values.index = pd.to_datetime(FRED_set["DATE"])
    values = values.resample(frequency).mean().dropna()
    #values['Symbol'] = x
    values = values.loc[~values.index.duplicated(keep='last')]
    #values = values.reset_index()

    FRED_pvt = pd.concat([FRED_pvt,values], axis=1)

#FRED_ = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
#FRED_.to_csv("/mnt/distvol/FRED_set.csv")

In [ ]:
#FRED_pvt = pd.pivot_table(FRED_, index=['DATE'])
FRED_pvt.to_csv(re.sub('code', 'data', wd)+"\FRED_pvt.csv")

#print(len(FRED_.columns))
#print(len(FRED_))
#FRED_pvt.columns = FRED_completed
#FRED_pvt.describe()

In [ ]:
#FRED_pvt.set_index(FRED_pvt.index)

In [ ]:
combined_set = pd.concat([FRED_pvt.set_index(FRED_pvt.index),commodities_pvt],axis=1)

In [ ]:
if True:
    combined_set = combined_set.interpolate(method='linear', limit_direction='forward', axis=0)

In [ ]:
combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<(len(combined_set)-1)]

In [ ]:
drops = combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<(len(combined_set)*.99)]
print(drops)
filtered = combined_set.columns.tolist()

for d in drops:
    #print(d)
    filtered.remove(d)
#filtered.remove(drops.tolist())
#combined_set[filtered].to_csv("/mnt/distvol/combined_set.csv")

#drop first/last row
all_data = combined_set[filtered]
all_data = all_data.iloc[:-1,:]
all_data = all_data.iloc[1:,:]

#all_data.loc[:, (all_data != all_data.iloc[0]).any()] 
#*** should fix duplications
all_data = all_data.loc[:,~all_data.columns.duplicated()]

all_data.to_csv(re.sub('code', 'data', wd)+"\combined_set.csv",index=True, index_label='Date')
#filtered
all_data.describe()

In [ ]:
all_data.index

In [ ]:
truncatedData = all_data.reindex(early.asfreq(frequency, method='pad').index).interpolate(method='time')
#len(truncatedData.dropna(axis=0).columns)
#truncatedData = truncatedData.replace([np.inf, -np.inf, np.NaN], 0).interpolate(method='time')

In [ ]:
len(truncatedData.columns)


In [ ]:
len(all_data.columns)

In [ ]:
len(truncatedData)

In [ ]:
deltas = truncatedData.dropna().pct_change().dropna()
deltas = deltas.replace([np.inf, -np.inf, np.NaN], 0)
#deltas = (truncatedData[all_data.columns]/truncatedData[all_data.columns].shift(-1))-1

In [ ]:
len(deltas)

In [ ]:
results = np.sum(deltas.isin([np.inf, -np.inf, np.NaN])).sort_values(kind="quicksort", ascending=True)
results[results>0]

In [ ]:
compare = 'DGS2'
#compare = '^SP500TR'
#compare = 'T10Y3M'
#compare = 'T10Y2Y'
target = "SPCS20RSA"
deltas[target] = deltas[target].diff(1).copy()
deltas = deltas.dropna().copy()
#target = '^SP500TR'
#target = pd.DataFrame(vetted_symbols).sample(n=1).values[0][0]
#target = etf_metals[0]
#target = crypto[1]
#target = '^GSPC'

pd.concat([truncatedData[compare].pct_change(),truncatedData[target].pct_change()],axis=1).dropna().corr()


In [ ]:

Training, Holdout = split_sequences(np.array(pd.DataFrame(deltas.index.strftime('%Y-%m-%d'))), 1009, 0)

In [ ]:
#train, valid = train_test_split(deltas.index,  test_size=0.3, random_state=0, shuffle=False)
#valid, test = train_test_split(valid,  test_size=0.5, random_state=0, shuffle=False)

train, valid = train_test_split(deltas.index,  test_size=0.3, random_state=0, shuffle=True)
valid, test = train_test_split(valid,  test_size=0.5, random_state=0, shuffle=True)

In [ ]:
from sklearn.model_selection import KFold
from scipy import stats # For in-built method to get PCC
import scipy

In [ ]:
#testing
lagn=9
p_threshold = .05
threshold = .5

num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=False)

final = pd.DataFrame()

#target = '^SP500TR'
#print()
#print(target)
#print()
#print(f"target: {target}")

#sets = range(0,len(Training),252)
#move this outside

X = deltas[set(deltas.columns).difference(target)].copy()
newX = pd.DataFrame()
y = pd.DataFrame(deltas[target].copy())

for m in X.columns:
    lagged = pd.DataFrame()
    newX = pd.concat([newX,X[m]],axis=1)
    
    for lag in range(1,lagn+1):
        temp = pd.DataFrame(X[m].shift(lag).copy())
        temp.columns = [m+"_"+str(lag)]
        lagged = pd.concat([lagged,temp],axis=1)
    #print(lagged)
    newX = pd.concat([newX,lagged],axis=1)

#newX = newX[set(newX.columns).difference(newX)].copy()
#newX = newX.dropna().copy()
#y = y.loc[newX.index]

kfold = KFold(n_splits=num_folds, shuffle=False)
kfold.get_n_splits(X)

In [ ]:
newX.columns

In [ ]:


#for m in X.columns:
#print(m)
#X_train = newX[newX.columns[newX.columns.str.contains(m)]].loc[train].copy()
#X_valid = newX[newX.columns[newX.columns.str.contains(m)]].loc[valid].copy()
X_train = newX.drop(X.columns, axis=1, inplace=False).loc[train].dropna().copy()
X_valid = newX.drop(X.columns, axis=1, inplace=False).loc[valid].dropna().copy()
X_test = newX.drop(X.columns, axis=1, inplace=False).loc[test].dropna().copy()

y_train = y.loc[X_train.index][target].copy()
y_valid = y.loc[X_valid.index][target].copy()
y_test = y.loc[X_test.index][target].copy()

exclude = ''

sig_table = np.zeros(shape=(len(all_data.columns)))
signs_table = np.zeros(shape=(len(all_data.columns)))

#this is for internal cross validation
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=num_folds)
train_ = []
test_ = []
for train_indices, test_indices in k_fold.split(X_train.index):
    train_.append(train_indices)
    test_.append(test_indices)


In [ ]:
'''
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(X_train)
pca.explained_variance_
print(pca.explained_variance_ratio_.cumsum())

#pca = PCA(n_components=1)
#pca.fit(X)
X_pca = pd.DataFrame(pca.transform(X_train))
X_pca.index = X_train.index
'''

In [ ]:
'''
from sklearn.ensemble import AdaBoostRegressor
from visuals import visuals as vs

# Train the supervised model on the training 
model = AdaBoostRegressor().fit(pd.DataFrame(X_pca), y_train)

# Extract the feature importances using .feature_importances_ 
importances = model.feature_importances_

# Plot
vs.feature_plot(importances, pd.DataFrame(X_pca), y_train)
'''

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sffs
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_boston

#lr = LinearRegression()

lr = LogisticRegression()

sffs1 = sffs(lr, 
          k_features=len(X_train)+1, 
          forward=True, 
          floating=True, 
          scoring='neg_mean_squared_error',
          #scoring='accuracy',
          n_jobs=-1,
          cv=5)

y_ = pd.DataFrame(np.where(y_train>0,1,0)))
sffs1 = sffs1.fit(X_train, pd.DataFrame(y_))


In [ ]:
fig = plot_sfs(sffs1.get_metric_dict(), kind='std_err')

plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()
print('Selected features:', sffs1.k_feature_idx_)

position = list(pd.DataFrame(sffs1.subsets_).loc["avg_score"]).index(pd.DataFrame(sffs1.subsets_).loc["avg_score"].max())

print(sffs1.subsets_[position]['feature_names'])

In [ ]:
EN_CV.get_params()

In [ ]:
features = list(pd.DataFrame(sffs1.subsets_).loc['feature_names'].values)

#feature_idx = pd.DataFrame(sfbs1.subsets_).loc['feature_idx'].tolist()
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet 
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LogisticRegressionCV



CV_results = pd.DataFrame()#[]#np.zeros(shape=(len(features),5))

for f in features:

    print(f)
    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1, )
    ratios = np.arange(0, 1, 0.05)
    alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]

    EN_CV = LogisticRegressionCV(l1_ratios=ratios, cv=cv, random_state=0, penalty='l2', n_jobs=-1)#.fit(X, y)
    #EN_CV = ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=cv, n_jobs=-1)
    # fit model

    X_subset = pd.DataFrame(X_train[list(np.asarray(f))])
    #y_ = pd.DataFrame(y_train)
    y_ = pd.DataFrame(np.where(y_train>0,1,0))
    EN_CV.fit(X_train, y_)
    
    
    #print('alpha: %f' % EN_CV.alpha_)
    #print('l1_ratio_: %f' % EN_CV.l1_ratio_)
    
    # evaluate model
    scores = cross_val_score(EN_CV, X_subset, y_, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    # force scores to be positive
    #scores = absolute(scores)
    #print('Mean MAE: %.3f (%.3f)' % (mean(scores[~np.isnan(scores)]), std(scores[~np.isnan(scores)])))
    
    #temp = pd.concat([pd.DataFrame({"features": [np.array(f)]}),pd.DataFrame(np.array(mean(scores[~np.isnan(scores)])).reshape(-1,1)),pd.DataFrame(np.array(std(scores[~np.isnan(scores)])).reshape(-1,1)),pd.DataFrame(np.array(EN_CV.alpha_).reshape(-1,1)),pd.DataFrame(np.array(EN_CV.l1_ratio_).reshape(-1,1))],axis=1)
    #logistic
    temp = pd.concat([pd.DataFrame({"features": [np.array(f)]}),pd.DataFrame(np.array(mean(scores[~np.isnan(scores)])).reshape(-1,1)),pd.DataFrame(np.array(std(scores[~np.isnan(scores)])).reshape(-1,1)),pd.DataFrame(np.array(EN_CV.l1_ratio_).reshape(-1,1))],axis=1)
    
    CV_results = pd.concat([CV_results,temp],axis=0)
                      



In [ ]:
sortedFeatures = list(np.array(pd.DataFrame(sffs1.subsets_).loc['feature_names'])[position])

In [ ]:
#CV_results.columns = ['features','mean error','std','alpha','lambda']
#logistic
CV_results.columns = ['features','mean error','std','lambda']
CV_results.index = np.array(range(0,len(features)))#range(1,len(features)+1)
plt.plot(CV_results['mean error'])
plt.show()

best = CV_results.iloc[CV_results['mean error'].idxmin()]
print(best)

parse = CV_results[CV_results['mean error']<=(np.min(CV_results['mean error'])+np.std(CV_results['mean error']))].iloc[0]
print(parse)

In [ ]:
print(model3.predict(X_test[parse['features']]))
print(np.array(np.where(y_test>0,1,0)))

In [ ]:
pd.concat([pd.concat([pd.DataFrame(np.where(y_valid<0,-1,1)),pd.DataFrame(model3.predict_proba(X_valid[parse['features']])).loc[:,1]],axis=1),pd.concat([pd.DataFrame(np.where(y_test<0,-1,1)),pd.DataFrame(model3.predict_proba(X_test[parse['features']])).loc[:,1]],axis=1)],axis=0)
#pd.DataFrame(model3.predict_proba(X_train[parse['features']])).loc[:,1]

In [ ]:
model3 = LogisticRegression(l1_ratio=parse['lambda'])
model3.fit(X_train[parse['features']],y_)
#print(MAPE(np.array(y_), model3.predict(X_train[parse['features']])))
print(mean(np.where([i[0] for i in np.where(y_==0,-1,1)]*np.where(model3.predict(X_train[parse['features']])==0,-1,1)<0,0,1)))
print(mean(np.where(np.where(y_valid<0,-1,1)*np.where(model3.predict(X_valid[parse['features']])==0,-1,1)<0,0,1)))
print(mean(np.where(np.where(y_test<0,-1,1)*np.where(model3.predict(X_test[parse['features']])==0,-1,1)<0,0,1)))

pd.concat([pd.DataFrame(np.where(y_valid<0,-1,1)),pd.DataFrame(model3.predict(X_valid[parse['features']]))],axis=1)
#print(MAPE(np.array(y_), model3.predict(X_train[parse['features']])))
#print(MAPE(np.array(np.where(y_valid>0,1,0)), model3.predict(X_valid[parse['features']])))
#print(MAPE(np.array(np.where(y_test>0,1,0)), model3.predict(X_test[parse['features']])))

In [ ]:
from sklearn.metrics import mean_squared_error

model0 = LinearRegression()
model1 = ElasticNet(alpha=parse['alpha'], l1_ratio=parse['lambda'])
model2 = ElasticNet(alpha=best['alpha'], l1_ratio=best['lambda'])


model0.fit(X_train[sortedFeatures],y_train)
model1.fit(X_train[parse['features']],y_train)
model2.fit(X_train[best['features']],y_train)

#linear
#parse
print(sortedFeatures)
#print("EN parse: " + str(mean_squared_error(y_valid, model1.predict(X_valid[parse['features']]))))
print(MAPE(y_train, model0.predict(X_train[sortedFeatures])))
print(MAPE(y_valid, model0.predict(X_valid[sortedFeatures])))
print(MAPE(y_test, model0.predict(X_test[sortedFeatures])))


#parse
print(parse['features'])
#print("EN parse: " + str(mean_squared_error(y_valid, model1.predict(X_valid[parse['features']]))))
print(MAPE(y_train,model1.predict(X_train[parse['features']])))
print(MAPE(y_valid, model1.predict(X_valid[parse['features']])))
print(MAPE(y_test, model1.predict(X_test[parse['features']])))

#best
#print("EN best: " + str(mean_squared_error(y_valid, model2.predict(X_valid[best['features']]))))
print(best['features'])
print(MAPE(y_train,model2.predict(X_train[best['features']])))
print(MAPE(y_valid, model2.predict(X_valid[best['features']])))
print(MAPE(y_test, model2.predict(X_test[best['features']])))


In [ ]:
exclude = ''

sig_table = np.zeros(shape=(len(sortedFeatures)))
signs_table = np.zeros(shape=(len(sortedFeatures)))

p_threshold = .05

from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=num_folds)
train_ = []
test_ = []
for train_indices, test_indices in k_fold.split(X_train.index):
    train_.append(train_indices)
    test_.append(test_indices)


In [ ]:
import pingouin as pg

sig_table = np.zeros(len(X_train[sortedFeatures].columns))
signs_table = np.zeros(len(X_train[sortedFeatures].columns))
significance = np.zeros(len(X_train[sortedFeatures].columns))
sign = np.zeros(len(X_train[sortedFeatures].columns))
purity = np.zeros(len(X_train[sortedFeatures].columns))

for it in range(0,len(train_)):
    max_pvalue = 1
    train_index = train_[it]
    test_index = test_[it]
    #display(all_data.iloc[test_index].describe())
    
    subset = pd.concat([X_train[sortedFeatures].iloc[train_index].loc[:, ~X_train[sortedFeatures].columns.isin([exclude])],y_train],axis=1)
    
    #skip y and states
    set_ = subset.loc[:, ~subset.columns.isin([target])].columns.tolist()
    
    n=len(subset)
    
    while(max_pvalue>=.05):

        dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
        p_values = pd.DataFrame(2*dist.cdf(-abs(subset.pcorr()[target]))).T
        p_values.columns = list(subset.columns)
        
        max_pname = p_values.idxmax(axis=1)[0]
        max_pvalue = p_values[max_pname].values[0]
        
        if (max_pvalue > .05):

            set_.remove(max_pname)
            temp = [target]
            temp.extend(set_)
            subset = subset[temp]
    
    winners = p_values.loc[:, ~p_values.columns.isin([target])].columns.tolist()
    sig_table = (sig_table + np.where(X_train[sortedFeatures].columns.isin(winners),1,0)).copy()
    signs_table[X_train[sortedFeatures].columns.get_indexer(winners)]+=np.where(subset.pcorr()[target][winners]<0,-1,1)
    
significance = pd.DataFrame(sig_table).T
significance.columns = list(X_train[sortedFeatures].columns)
display(significance)

sign = pd.DataFrame(signs_table).T
sign.columns = list(X_train[sortedFeatures].columns)
display(sign)

purity = abs((sign/num_folds)*(sign/significance)).T.replace([np.inf, -np.inf, np.NaN], 0)
display(purity.T)

threshold = .5

chosen = list(purity.T.columns.values[np.array(purity.T>threshold).reshape(len(X_train[sortedFeatures].columns,))])
display(chosen)    

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from zca import zca

zca = zca.ZCA()
#chosen = sortedFeatures
#chosen = parse['features']
chosen = best['features']

data_zca = zca.fit_transform(X_train[chosen])

#zca.fit(X_train[chosen])

#data_zca = zca.transform(X_train[chosen])

model = sm.OLS(y_train,X_train[chosen])
results = model.fit()

print(results.summary())

reg = LinearRegression().fit(X_train[chosen], y_train)
#reg.score(X, y)

#reg.coef_

#reg.intercept_

#import statsmodels as ssm
#X=sm.add_constant(X_train[sortedFeatures])        #to add constant value in the model
model= sm.OLS(y_train,X_train[chosen]).fit()         #fitting the model
summary = model.summary()      #summary of the model
display(summary)
print(MAPE(y_train,model.predict(X_train[chosen])))
print(MAPE(y_valid,model.predict(X_valid[chosen])))
print(MAPE(y_test,model.predict(X_test[chosen])))

In [ ]:
print(parse['alpha'], parse['lambda'])

In [ ]:
import statsmodels.stats.proportion as smp

correct = np.where(pd.concat([y_valid*model2.predict(X_valid[chosen]),y_test*model2.predict(X_test[chosen])],axis=0) > 0, 1 ,0)

ci_low, ci_upp = smp.proportion_confint(sum(correct), len(correct), alpha=0.05, method='normal')

print(ci_low)
print(ci_upp)

In [ ]:
mean(correct)

In [ ]:
#plt.scatter(y_train, model.predict(X_train[sortedFeatures]))
plt.scatter(y_valid, model.predict(X_valid[sortedFeatures]))
plt.scatter(y_test, model.predict(X_test[sortedFeatures]))


In [ ]:
#testing
lagn=9
p_threshold = .05
threshold = .5

num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=False)

final = pd.DataFrame()

target = "SPCS20RSA"
#print()
#print(target)
#print()
#print(f"target: {target}")

#sets = range(0,len(Training),252)
#move this outside
X_train = deltas.loc[train][set(deltas.columns).difference(target)].copy()
X_valid = deltas.loc[test][set(deltas.columns).difference(target)].copy()
y_train = deltas.loc[train][target].copy()
y_valid = deltas.loc[test][target].copy()

#X_train = deltas.loc[train][X_train.columns[~X_train.columns.isin([target])]].copy()
#X_valid = deltas.loc[test][X_train.columns[~X_train.columns.isin([target])]].copy()
#y_train = deltas.loc[train][target].copy()
#y_valid = deltas.loc[test][target].copy()

kfold = KFold(n_splits=num_folds, shuffle=False)
kfold.get_n_splits(X_train)

for m in X_train.columns:
    #print(m)

    corrs = []
    ps = []
    lags = []

    sig_table = np.zeros(shape=(2))
    signs_table = np.zeros(shape=(2))

    sets = np.zeros(shape=(num_folds,lagn))

    iterator = 0

    for train_index, test_index in kfold.split(X_train):

        subsetX = X_train.iloc[train_index]
        subsetY = y_train.iloc[train_index]

        #skip y and states

        n=len(subsetX)

        temp = pd.concat([subsetX[m].shift((lag-lagn)[0]),subsetY], axis=1).dropna()

        dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)

        chosen = pd.DataFrame()

        #train
        setc = crosscorrelation(np.array(temp[m]),np.array(temp.iloc[:,1]), lagn)
        p_values = pd.DataFrame(2*dist.cdf(-abs(setc))).T

        fields = np.argwhere(np.array(p_values)[0] <= p_threshold)

        #print(fields[fields>lagn+1]-lagn-1)
        #print(setc[fields[fields>lagn+1]])

        #sets[iterator] = (np.where(p_values <= p_threshold, True, 0)*setc)[0][lagn+1:]
        sets[iterator] = (np.array(p_values)*setc)[0][lagn+1:]             

        iterator = iterator + 1

    csets_avg = np.sum(sets, axis=0)/num_folds       

    lag = pd.DataFrame(abs(csets_avg)).idxmax()
    #print(f"correlation: {setc[lag]}")

    #pearson_coef, p_value = stats.pearsonr(np.array(temp.iloc[:,0]),np.array(temp.iloc[:,1])) #define the columns to perform calculations on

    corrs.append(csets_avg[lag])
    p_value = (2*dist.cdf(-abs(csets_avg[lag])))
    ps.append(p_value)
    lags.append(lag)    

    print(p_value)
    if (p_value < .05):
        for p in range(0,len(sets)):
            plt.plot(sets[p])   
        plt.show()

        print(target)
        print(m)
        #print(i)
        #print(f"lag: {lag-lagn}")
        #print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value) # Results
        temp1 = pd.DataFrame([target, m,(lag-lagn)[0],pearson_coef]).T
        temp1.columns = ["target","name","lag","pearson"]

        #p_values = pd.DataFrame(2*dist.cdf(-abs(temp.corr()[target]))).T
        #p_values.columns = list(temp.columns)

        #if(p_values[p_values.loc[:, ~p_values.columns.isin([target])].columns.tolist()]<=p_threshold):
        #winners = p_values.loc[:, ~p_values.columns.isin([target])].columns.tolist()
        #winners = m

        #sig_table = (sig_table + np.where(temp.columns.isin([winners]),1,0)).copy()
        #print(sig_table)
        #signs_table[temp.columns.get_indexer([winners])]+=np.where(temp.corr()[target][winners]<0,-1,1)                 
        #chosen = pd.concat([temp1,chosen],axis=0)
        chosen.append(m)

        c_threshold = dist.ppf(.05)
        plt.plot(np.sum(sets, axis=0)/num_folds)
        plt.axhline(y = 0, color = 'r', linestyle = '-')
        plt.axhline(y = c_threshold, color = 'y', linestyle = '-')
        plt.axhline(y = abs(c_threshold), color = 'y', linestyle = '-')

        plt.show()
        x=range(0,lagn)
        plt.stackplot(x,sets[0],sets[1],sets[2],sets[3],sets[4], labels=['A','B','C','D','E'])
        plt.axhline(y = 0, color = 'r', linestyle = '-')
        plt.axhline(y = c_threshold, color = 'y', linestyle = '-')
        plt.axhline(y = abs(c_threshold), color = 'y', linestyle = '-')        
        plt.show()

    #significance = pd.DataFrame(sig_table).T
    #significance.columns = list(temp.columns)
    #print(significance)

    #sign = pd.DataFrame(signs_table).T
    #sign.columns = list(temp.columns)

    #purity = abs((sign/num_folds)*(sign/significance)).T.replace([np.inf, -np.inf, np.NaN], 0)
    #print(purity)

    #ichosen = list(purity.T.columns.values[np.array(purity.T>=threshold).reshape(len(temp.columns,))])
    #print(ichosen)
    #chosen.append(ichosen)

    #print(chosen)

    #print(lags)

    #print(m)
    #print(ps)          

    for i in range(0,len(chosen)):
        values = chosen.reset_index().iloc[i]
        name = values['name']
        target = values['target']
        lag = values['lag']
        #print(lag)
        aggregate = pd.DataFrame()

        #test
        innerSet = pd.concat([X_valid[name].shift(lag),y_valid],axis=1).dropna()

        for p in range(0,len(innerSet)):     
            #print(innerSet.iloc[p][name] )
            if(innerSet.iloc[p][name] < 0):
                d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),innerSet.iloc[p][target], np.nan, 'l'])
                #print("lower")
            elif (innerSet.iloc[p][name] > 0):
                d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),np.nan, innerSet.iloc[p][target], 'u'])
                #print("upper")
            else:
                d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),np.nan, np.nan, np.nan])

            aggregate = pd.concat([d.T,aggregate],axis=0)
        #print(aggregate.loc[:,['l','u']].replace([np.inf, -np.inf, np.NaN], 0).cumsum().iloc[-1])

        aggregate.columns = "Date","l","u","class"

        pearson_coef, p_value = stats.pearsonr(np.array(innerSet.iloc[:,0]),np.array(innerSet.iloc[:,1])) #define the columns to perform calculations on

        #print(f"name: {name}", f"lag: {lag}", f"corr: {innerSet.corr().iloc[0][1]}")
        if p_value < .05 and (values['pearson']*pearson_coef) > 0:

                newData = pd.concat([pd.DataFrame(innerSet[target]).set_index(innerSet[target].index.strftime('%Y-%m-%d')),aggregate.set_index("Date")],axis=1).replace([np.inf, -np.inf, np.NaN], 0)

                if newData[[target,"l","u"]].cumsum().iloc[-1][target] < newData[[target,"l","u"]].cumsum().iloc[-1]['l'] or newData[[target,"l","u"]].cumsum().iloc[-1][target] < newData[[target,"l","u"]].cumsum().iloc[-1]['u']:
                    print(f"target: {target}")
                    print(f"name: {name}", f"lag: {lag}", f"corr: {pearson_coef}", f"p-value: {p_value}")
                    print(pd.DataFrame(newData[[target,"l","u"]].cumsum().iloc[-1]).T)

                    x_ticks = newData.index[np.arange(0, len(newData.index), 200)]

                    plt.plot(newData[[target,"l","u"]].cumsum())
                    plt.legend(loc='upper left', fancybox=True, ncol=5, labels=[target,"l","u"])
                    #plt.legend(loc="upper left",fontsize=8)

                    plt.xticks(x_ticks, rotation = 45)
                    plt.show()

                    temp1 = pd.DataFrame([target, name, pearson_coef, p_value, lag, max(newData[[target,"l","u"]].cumsum().iloc[-1][['l','u']]), newData[[target]].cumsum().iloc[-1][target] ]).T
                    final = pd.concat([temp1,final],axis=0)

final.columns = ["target","correlate","pearson","p-value","lag","TCR","holdTCR"]


In [ ]:
#testing
lagn=1
p_threshold = .05
threshold = .5

num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=False)

final = pd.DataFrame()

for target in set(deltas.columns) & set(vetted_symbols):
    #print()
    #print(target)
    #print()
    #print(f"target: {target}")
    
    #sets = range(0,len(Training),252)
    #move this outside
    X_train = deltas.loc[train][X_train.columns[~X_train.columns.isin([target])]].copy()
    X_valid = deltas.loc[test][X_train.columns[~X_train.columns.isin([target])]].copy()
    y_train = deltas.loc[train][target].copy()
    y_valid = deltas.loc[test][target].copy()
    
    kfold = KFold(n_splits=num_folds, shuffle=False)
    kfold.get_n_splits(X_train)

    for m in X_train.columns:
        #print(m)
        
        corrs = []
        ps = []
        lags = []

        sig_table = np.zeros(shape=(2))
        signs_table = np.zeros(shape=(2))

        sets = np.zeros(shape=(num_folds,lagn))
        
        iterator = 0
        
        for train_index, test_index in kfold.split(X_train):

            subsetX = X_train.iloc[train_index]
            subsetY = y_train.iloc[train_index]

            #skip y and states

            n=len(subset)
            
            temp = pd.concat([subsetX[m].shift((lag-lagn)[0]),subsetY], axis=1).dropna()

            dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)

            chosen = pd.DataFrame()

            #train
            setc = crosscorrelation(np.array(temp[m]),np.array(temp.iloc[:,1]), lagn)
            p_values = pd.DataFrame(2*dist.cdf(-abs(setc))).T
            
            fields = np.argwhere(np.array(p_values)[0] <= p_threshold)
           
            #print(fields[fields>lagn+1]-lagn-1)
            #print(setc[fields[fields>lagn+1]])
            
            #sets[iterator] = (np.where(p_values <= p_threshold, True, 0)*setc)[0][lagn+1:]
            sets[iterator] = (np.array(p_values)*setc)[0][lagn+1:]             
            
            iterator = iterator + 1
        
        csets_avg = np.sum(sets, axis=0)/num_folds       
        
        lag = pd.DataFrame(abs(csets_avg)).idxmax()
        #print(f"correlation: {setc[lag]}")

        #pearson_coef, p_value = stats.pearsonr(np.array(temp.iloc[:,0]),np.array(temp.iloc[:,1])) #define the columns to perform calculations on

        corrs.append(csets_avg[lag])
        p_value = (2*dist.cdf(-abs(csets_avg[lag])))
        ps.append(p_value)
        lags.append(lag)    

        #print(p_value)
        if (p_value < .05):
            for p in range(0,len(sets)):
                plt.plot(sets[p])   
            plt.show()
            
            print(target)
            print(m)
            #print(i)
            #print(f"lag: {lag-lagn}")
            #print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value) # Results
            temp1 = pd.DataFrame([target, m,(lag-lagn)[0],pearson_coef]).T
            temp1.columns = ["target","name","lag","pearson"]

            #p_values = pd.DataFrame(2*dist.cdf(-abs(temp.corr()[target]))).T
            #p_values.columns = list(temp.columns)

            #if(p_values[p_values.loc[:, ~p_values.columns.isin([target])].columns.tolist()]<=p_threshold):
            #winners = p_values.loc[:, ~p_values.columns.isin([target])].columns.tolist()
            winners = m

            #sig_table = (sig_table + np.where(temp.columns.isin([winners]),1,0)).copy()
            #print(sig_table)
            #signs_table[temp.columns.get_indexer([winners])]+=np.where(temp.corr()[target][winners]<0,-1,1)                 
            #chosen = pd.concat([temp1,chosen],axis=0)
            chosen.append(m)

            c_threshold = dist.ppf(.05)
            plt.plot(np.sum(sets, axis=0)/num_folds)
            plt.axhline(y = 0, color = 'r', linestyle = '-')
            plt.axhline(y = c_threshold, color = 'y', linestyle = '-')
            plt.axhline(y = abs(c_threshold), color = 'y', linestyle = '-')

            plt.show()
            x=range(0,lagn)
            plt.stackplot(x,sets[0],sets[1],sets[2],sets[3],sets[4], labels=['A','B','C','D','E'])
            plt.axhline(y = 0, color = 'r', linestyle = '-')
            plt.axhline(y = c_threshold, color = 'y', linestyle = '-')
            plt.axhline(y = abs(c_threshold), color = 'y', linestyle = '-')        
            plt.show()

        #significance = pd.DataFrame(sig_table).T
        #significance.columns = list(temp.columns)
        #print(significance)

        #sign = pd.DataFrame(signs_table).T
        #sign.columns = list(temp.columns)

        #purity = abs((sign/num_folds)*(sign/significance)).T.replace([np.inf, -np.inf, np.NaN], 0)
        #print(purity)

        #ichosen = list(purity.T.columns.values[np.array(purity.T>=threshold).reshape(len(temp.columns,))])
        #print(ichosen)
        #chosen.append(ichosen)
        
        #print(chosen)
        
        #print(lags)
        
        #print(m)
        #print(ps)          
        
        for i in range(0,len(chosen)):
            values = chosen.reset_index().iloc[i]
            name = values['name']
            target = values['target']
            lag = values['lag']
            #print(lag)
            aggregate = pd.DataFrame()

            #test
            innerSet = pd.concat([X_valid[name].shift(lag),y_valid],axis=1).dropna()

            for p in range(0,len(innerSet)):     
                #print(innerSet.iloc[p][name] )
                if(innerSet.iloc[p][name] < 0):
                    d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),innerSet.iloc[p][target], np.nan, 'l'])
                    #print("lower")
                elif (innerSet.iloc[p][name] > 0):
                    d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),np.nan, innerSet.iloc[p][target], 'u'])
                    #print("upper")
                else:
                    d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),np.nan, np.nan, np.nan])

                aggregate = pd.concat([d.T,aggregate],axis=0)
            #print(aggregate.loc[:,['l','u']].replace([np.inf, -np.inf, np.NaN], 0).cumsum().iloc[-1])

            aggregate.columns = "Date","l","u","class"

            pearson_coef, p_value = stats.pearsonr(np.array(innerSet.iloc[:,0]),np.array(innerSet.iloc[:,1])) #define the columns to perform calculations on

            #print(f"name: {name}", f"lag: {lag}", f"corr: {innerSet.corr().iloc[0][1]}")
            if p_value < .05 and (values['pearson']*pearson_coef) > 0:

                    newData = pd.concat([pd.DataFrame(innerSet[target]).set_index(innerSet[target].index.strftime('%Y-%m-%d')),aggregate.set_index("Date")],axis=1).replace([np.inf, -np.inf, np.NaN], 0)

                    if newData[[target,"l","u"]].cumsum().iloc[-1][target] < newData[[target,"l","u"]].cumsum().iloc[-1]['l'] or newData[[target,"l","u"]].cumsum().iloc[-1][target] < newData[[target,"l","u"]].cumsum().iloc[-1]['u']:
                        print(f"target: {target}")
                        print(f"name: {name}", f"lag: {lag}", f"corr: {pearson_coef}", f"p-value: {p_value}")
                        print(pd.DataFrame(newData[[target,"l","u"]].cumsum().iloc[-1]).T)

                        x_ticks = newData.index[np.arange(0, len(newData.index), 200)]

                        plt.plot(newData[[target,"l","u"]].cumsum())
                        plt.legend(loc='upper left', fancybox=True, ncol=5, labels=[target,"l","u"])
                        #plt.legend(loc="upper left",fontsize=8)

                        plt.xticks(x_ticks, rotation = 45)
                        plt.show()

                        temp1 = pd.DataFrame([target, name, pearson_coef, p_value, lag, max(newData[[target,"l","u"]].cumsum().iloc[-1][['l','u']]), newData[[target]].cumsum().iloc[-1][target] ]).T
                        final = pd.concat([temp1,final],axis=0)

final.columns = ["target","correlate","pearson","p-value","lag","TCR","holdTCR"]


In [ ]:
final

In [ ]:
results2 = pd.DataFrame()

for i in range(0,len(final)):
    values = final.reset_index().iloc[i]
    name = values['correlate']
    target = values['target']
    lag = values['lag']
    #print(lag)
    aggregate = pd.DataFrame()

    X_test = deltas.loc[test][set(deltas.columns).difference(target)]
    y_test = deltas.loc[test][target]
    
    #test
    innerSet = pd.concat([X_test[name].shift(lag),y_test],axis=1).dropna()

    for p in range(0,len(innerSet)):     
        #print(innerSet.iloc[p][name] )
        if(innerSet.iloc[p][name] < 0):
            d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),innerSet.iloc[p][target], np.nan, 'l'])
            #print("lower")
        elif (innerSet.iloc[p][name] > 0):
            d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),np.nan, innerSet.iloc[p][target], 'u'])
            #print("upper")
        else:
            d = pd.DataFrame([innerSet.index[p].strftime('%Y-%m-%d'),np.nan, np.nan, np.nan])

        aggregate = pd.concat([d.T,aggregate],axis=0)
    #print(aggregate.loc[:,['l','u']].replace([np.inf, -np.inf, np.NaN], 0).cumsum().iloc[-1])

    aggregate.columns = "Date","l","u","class"

    pearson_coef, p_value = stats.pearsonr(np.array(innerSet.iloc[:,0]),np.array(innerSet.iloc[:,1])) #define the columns to perform calculations on

    #print(f"name: {name}", f"lag: {lag}", f"corr: {innerSet.corr().iloc[0][1]}")
    if p_value < .05:

            newData = pd.concat([pd.DataFrame(innerSet[target]).set_index(innerSet[target].index.strftime('%Y-%m-%d')),aggregate.set_index("Date")],axis=1).replace([np.inf, -np.inf, np.NaN], 0)

            if newData[[target,"l","u"]].cumsum().iloc[-1][target] < newData[[target,"l","u"]].cumsum().iloc[-1]['l'] or newData[[target,"l","u"]].cumsum().iloc[-1][target] < newData[[target,"l","u"]].cumsum().iloc[-1]['u']:
                print(f"target: {target}")
                print(f"name: {name}", f"lag: {lag}", f"corr: {pearson_coef}", f"p-value: {p_value}")
                print(pd.DataFrame(newData[[target,"l","u"]].cumsum().iloc[-1]).T)

                x_ticks = newData.index[np.arange(0, len(newData.index), 200)]

                plt.plot(newData[[target,"l","u"]].cumsum())
                plt.legend(loc='upper left', fancybox=True, ncol=5, labels=[target,"l","u"])
                #plt.legend(loc="upper left",fontsize=8)

                plt.xticks(x_ticks, rotation = 45)
                plt.show()

                temp1 = pd.DataFrame([target, name, pearson_coef, p_value, lag, max(newData[[target,"l","u"]].cumsum().iloc[-1][['l','u']]), newData[[target]].cumsum().iloc[-1][target] ]).T
                results2 = pd.concat([temp1,results2],axis=0)
                
results2.columns = ["target","correlate","pearson","p-value","lag","TCR","holdTCR"]                

In [ ]:
len(final)

In [ ]:
results2

In [ ]:
pd.concat([final['pearson'],results2['pearson']],axis=1)

In [ ]:
pd.concat([final['pearson']*results2['pearson']],axis=1)

In [ ]:
aggregate = pd.DataFrame()

for s in Training:
    #print(s)
    set_dates = s
    #print(set_dates[1])
    #print(set_dates[-1])
    
    #rate of change compare
    filtered2 =  pd.concat([truncatedData.loc[[i[0] for i in set_dates]][compare],truncatedData.loc[[i[0] for i in set_dates]][target]],axis=1).pct_change().dropna().replace([np.inf, -np.inf, np.NaN], 0)
    #quantiles2 = filtered2[compare].quantile(q=[0, .25, .5, .75, 1], interpolation='linear')

    #prop_cycle = plt.rcParams['axes.prop_cycle']
    #colors = cycle(prop_cycle.by_key()['color'])
    
    #subset, Holdout = split_sequences(np.array(pd.DataFrame(filtered2.index.strftime('%Y-%m-%d'))), 126, 0)
    
    #for i in subset:
        
    #quantiles = pd.DataFrame(filtered2.iloc[0:-2][compare].quantile([0,.5,1]),columns=['min','median','max'])
    #np.percentile((filtered2.iloc[0:-2][compare]),50)
    
    t = pd.DataFrame(filtered2.iloc[0:-3][compare].quantile([0,.5,1])).T
    t.columns = ['min','median','max']
    quantiles = t.reset_index(drop=True)

    #lower = filtered2.iloc[0:-1][(filtered2.iloc[0:-2][compare]<=quantile)].index
    #upper = ~filtered2.iloc[0:-2].index.isin(lower)



    #my_dpi = 100
    #fig, axes = plt.subplots(figsize=(12, 4),ncols=3, nrows=1)
    #ax1, ax2, ax3 = axes.ravel()
    #sns.set(style="ticks")
    #sns.despine(fig=fig)

    #dataframe = filtered2.loc[dates]
    #returns = dataframe[target].dropna()

    #l = np.where(filtered2.iloc[0:-2][compare] <= quantile, filtered2.iloc[-1][target], 0)
    #u = np.where(filtered2.iloc[upper][compare] > quantile, filtered2.iloc[-1][target], 0)
    if((filtered2.iloc[-2][compare] <= quantiles['median'][0])):
        d = pd.DataFrame([filtered2.iloc[-1][target], np.nan, 'l'])
    else:
        d = pd.DataFrame([np.nan, filtered2.iloc[-1][target], 'u'])
    #d = pd.concat([pd.DataFrame(l.tolist()),pd.DataFrame(u.tolist())],axis=1)
    #print(len(d))
    #print(d)
    
    #print(pd.concat([d.T, quantiles.T.reset_index(drop=True)],axis=1))
    #aggregate = pd.concat([d.T, quantiles.T.reset_index(drop=True), aggregate],axis=0)
    aggregate = pd.concat([pd.concat([d.T, quantiles],axis=1),aggregate],axis=0)
    #print(aggregate)
    #print(len(aggregate))
    #print(aggregate)
    #d.index = filtered2.iloc[-1].name.strftime('%Y-%m-%d')
   
    #sns.regplot(x=compare, y=target, data=dataframe, ax=ax1)

    #sns.violinplot(x=dataframe[compare].dropna(),color=next(colors), ax=ax2)

    #sns.vioinplot(x=returns,color=next(colors), ax=ax3) 
    #plt.show()




In [ ]:
len(Training)

aggregate.columns = ['l','u','class','min','median','max']

dates = []
for item in Training:
    last = item[-1]
    dates.append(last[0])
    
aggregate.loc[:,['l','u']].replace([np.inf, -np.inf, np.NaN], 0).cumsum().iloc[-1]

aggregate.index=dates

#print(aggregate.cumsum().iloc[-1])
#aggregate.reset_index()
#ax = sns.regplot(x=aggregate.columns[1], y='index', data=aggregate.reset_index())

x_ticks = aggregate.index[np.arange(0, len(aggregate.index), 200)]

plt.plot(aggregate.loc[:,['l']].replace([np.inf, -np.inf, np.NaN], 0).cumsum())

plt.xticks(x_ticks, rotation = 45)
plt.show()

In [ ]:
temp1 = pd.concat([truncatedData[compare].pct_change(),truncatedData[target]],axis=1).loc[dates]
temp2 = pd.concat([truncatedData[compare].pct_change(),truncatedData[target].pct_change()],axis=1).loc[dates]
temp1.index = temp1.index.copy().strftime('%Y-%m-%d')
temp2.index = temp2.index.copy().strftime('%Y-%m-%d')
temp = pd.concat([temp1,temp2,aggregate],axis=1)
newNames = [compare,target,compare+"_pct_change",target+"_pct_change"]
newNames.extend(temp.columns[4:10])
temp.columns = newNames

temp.to_csv('../data/processed/output.csv', index=True) 

In [ ]:
from sklearn import metrics
#scores = np.where(temp[compare] > 1, 1, 0)

#idk why this library uses 2, 1 vs 1 and 0
fpr, tpr, thresholds = metrics.roc_curve(np.where(temp[target+"_pct_change"] > 0, 2, 1), np.array(temp[compare+"_pct_change"].shift(-1)) , pos_label=2)

In [ ]:
metrics.plot_roc_curve(clf, X_test, y_test)  

In [ ]:
from sklearn.metrics import roc_curve
#plt.plot(fpr,tpr, label='AUC = ' + str(round(roc_auc_score(y,m.oob_decision_function_[:,1]), 2)))
#plt.legend(loc='lower right')

from plot_metric.functions import BinaryClassification
# Visualisation with plot_metric
bc = BinaryClassification(np.where(temp[target+"_pct_change"] > 0, 1, 0), np.array(temp[compare+"_pct_change"]), labels=["Class 1", "Class 2"])

# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.show()

In [ ]:


print("optimal cutoff")
#https://stackoverflow.com/questions/28719067/roc-curve-and-cut-off-point-python
Find_Optimal_Cutoff(np.where(temp[target+"_pct_change"] > 0, 1, 0), np.array(temp[compare+"_pct_change"]))


In [ ]:
from kneed import DataGenerator, KneeLocator
kneedle = KneeLocator(fpr, tpr, S=1.0, curve="concave", direction="increasing")
#kneedle.plot_knee_normalized()
kneedle.plot_knee()

In [ ]:
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]


In [ ]:
plt.scatter(aggregate[aggregate['bifur']=='l'].iloc[:,0].cumsum(),  truncatedData[target].pct_change().loc[aggregate[aggregate['bifur']=='l'].iloc[:,0].index.tolist()])
plt.show()
aggregate[aggregate['bifur']=='l'].iloc[:,0].hist()
plt.show()

In [ ]:
plt.scatter(aggregate[aggregate['bifur']=='u'].iloc[:,1].cumsum(),  truncatedData[target].pct_change().loc[aggregate[aggregate['bifur']=='u'].iloc[:,1].index.tolist()])
plt.show()
aggregate[aggregate['bifur']=='u'].iloc[:,1].hist()
plt.show()

In [ ]:
#252 trading days a year
#30 days = 21
#60 = 42
#90 = 63

#return = current - prior / prior

#for i in range(1,63): #[1,21,42,63]
for i in [1,21,42,63,84]: #[1,21,42,84]
    print(i)
    rate_of_change = (truncatedData[compare]-truncatedData[compare].shift(i))/truncatedData[compare].shift(i)
    
    newDF = pd.concat([rate_of_change,truncatedData[target].pct_change()],axis=1).dropna()

    set1 = newDF[compare]
    set2 = newDF[target]
    
    lags = range(0,85) #[0,21,42,63]
    mcorrs_ = []
    mlags = []
    for j in lags:
        
        newSet = pd.concat([set1.shift(j),set2],axis=1).dropna()
      
        mcorrs_.append(np.array(newSet.corr())[1,0])
        
    plt.plot(mcorrs_)
    plt.show()





In [ ]:
pd.concat([(truncatedData[compare]-truncatedData[compare].shift(1))/truncatedData[compare].shift(1),truncatedData[target].pct_change().shift(0)],axis=1).dropna().corr()

In [ ]:
plt.scatter((truncatedData[compare]-truncatedData[compare].shift(1))/truncatedData[compare].shift(1),truncatedData[target].pct_change().shift(0))

In [ ]:
#import matplotlib.pyplot as plt
#plt.matshow(
df = combined_set.loc[combined_set.index>=start_date.strftime('%Y-%m-%d')]

r_ = df.corr()

filter = r_[compare]
                       
filter = filter[filter<1]
filter = filter.sort_values(kind="quicksort", ascending=True)
print(filter.head(10))
print(filter.tail(10))
